In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt #for creating static, animated and interactive visulization
import seaborn as sns #data visulation library used for making statistical graphics

import warnings

In [ ]:
#readind data
df = pd.read_csv('../input/heart-failure-prediction/heart.csv')

In [ ]:
#size (rows,columns)
df.shape

In [ ]:
#viewing data
df.head()

In [ ]:
cleaned_df = df.drop(['Sex', 'RestingECG', 'ST_Slope', 'ChestPainType', 'ExerciseAngina'], axis=1)

In [ ]:
cleaned_df.sample(5)

In [ ]:
cleaned_df.shape

In [ ]:
sns.pairplot(data=df, hue='Sex')

In [ ]:
# add histograms to diagonals of Seaborn pairplot
sns.pairplot(data=df, hue='Sex', diag_kind='hist')

In [ ]:
cor = df.corr()

cor

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(cor, vmax=1, square=True, annot=True)
plt.title('Correlation between features of Heart Dieses')

**Plotting Heart Rate ratio by Age**

In [ ]:
sns.lmplot(x='Age', y='MaxHR', data=df, hue='Sex', fit_reg=False)

In [ ]:
#NORMALIZATION 
# reduce dimensions with PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

#cleaned_df = pd.DataFrame(scaler.fit_transform(cleaned_df), columns=cleaned_df.columns, index= cleaned_df.index)


In [ ]:
cleaned_df.head()

In [ ]:

pca = PCA(2) # how many Principle components to retain.

out_pca = pca.fit_transform(cleaned_df[cleaned_df.columns.tolist()])

pca_df= pd.DataFrame(data=out_pca, columns=['Pca1', 'Pca2']) # as we have retained 2 PCs so we wrote pc1 and pc2 only.

print(pca_df)

In [ ]:
# total_var = pca.explained_variance_ratio_.sum() * 100
sns.lmplot(x='Pca1', y='Pca2', data=pca_df, fit_reg=False)


In [ ]:
# cluster With k-means and check silhouette score
from sklearn.cluster import KMeans

def kmns_clustring(data, clusters):
    # initialize k-means algo object
    kmeans = KMeans(n_clusters =clusters,random_state=42 )
    kmeans.fit(data)
    
    kmeans.clusters = kmeans.cluster_centers_
    # fit algo to pca and find silhouette score
    kmeans.y_kmn =  kmeans.fit_predict(out_pca)
    
    return kmeans



In [ ]:
kmeans = kmns_clustring(data=pca_df, clusters=8)

In [ ]:
plt.figure(figsize=(15,8))
def scatter_plot(kmns, df):
    for i in np.unique(kmns.y_kmn):
        plt.scatter(out_pca[kmns.y_kmn == i ,0], out_pca[kmns.y_kmn == i, 1], s=50)
        plt.scatter(kmns.clusters[i][0] , kmns.clusters[i][1] , s = 80,marker='+', color = 'k')
    plt.show()
scatter_plot(kmeans, pca_df)

In [ ]:
 ##from sklearn.metrics import silhouette_score
 ##  def s_score(dataset):
 ## silhouette_coefficient = []
 ##  for k in range(2,11):
   ## kmeans = KMeans(n_clusters = k, init='k-means++', n_init=30, max_iter=500, random_state=42 )
    ##    kmeans.fit(dataset)
     ##  score = silhouett_score(dataset, kmeans.labels_)
  ## silhouette_coefficient.append(score)
 ## return silhouette_coefficient
 ##  s_score = s_scorer(dataset=pca_df)

In [ ]:
##plt.style.use('fivethirtyeight')
##plt.plot(range(1,9), s_sccore)
##plt.xticks(range(1,9))
######plt.xlabel('Number of Clusters')
##plt.ylabel('Error')
##plt.show()

In [ ]:
X=df.iloc[:,[0,2,3,4,5,6,7,8,9,10,11]]

In [ ]:
X

In [ ]:
Y=df.iloc[:,1]
Y

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_x=LabelEncoder()
X=X.apply(LabelEncoder().fit_transform)
X

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [ ]:
## classify with decision tree
from sklearn.tree import DecisionTreeClassifier
regressor=DecisionTreeClassifier(random_state=1)
# fit the model
regressor.fit(X_train, y_train)

In [ ]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

In [ ]:

y_pred= regressor.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
# check prediction score
print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
##plt.figure(figsize=(12,8))

from sklearn import tree

tree.plot_tree(regressor.fit(X_train, y_train))

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(regressor, out_file=None, 
                             feature_names=X_train.columns,  
                            class_names=y_train,  
                            filled=True, rounded=True,  
                             special_characters=True)

graph = graphviz.Source(dot_data) 

graph 

In [ ]:
from sklearn.model_selection import train_test_split
def naive_bayes_accuracy(dataframe, predicted_y):
    # splitting X and y into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(dataframe, predicted_y, test_size=0.4, random_state=1, stratify=y)

In [ ]:
 # training the model on training set
     from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)

In [ ]:
# making predictions on the testing set
    y_pred = gnb.predict(X_test)

In [ ]:
# comparing actual response values (y_test) with predicted response values (y_pred)
    from sklearn import metrics
    score = metrics.accuracy_score(y_test, y_pred)*100
    return score
print('K-MEAN Accuracy: %.3f' %  naive_bayes_accuracy(df, kmeans.y_kmn))